# Preprocessamento

In [1]:
import pandas as pd
import numpy as np

In [2]:
diretorio_dados = "dados/cru"
url_proposicoes = "http://dadosabertos.camara.leg.br/arquivos/proposicoes/{formato}/proposicoes-{ano}.{formato}"
url_temas = "http://dadosabertos.camara.leg.br/arquivos/proposicoesTemas/{formato}/proposicoesTemas-{ano}.{formato}"
url_autores = "http://dadosabertos.camara.leg.br/arquivos/proposicoesAutores/{formato}/proposicoesAutores-{ano}.{formato}"
formato = "csv"
anos = range(1998, 2023)

diretorio_dados_processados = "dados/processado"
filtro_colunas_proposicoes = ['siglaTipo', 'ano',
       'ementa', 'ementaDetalhada', 'keywords', 'dataApresentacao',
       'ultimoStatus_dataHora',  'ultimoStatus_descricaoTramitacao',
                              'ultimoStatus_descricaoSituacao']
filtro_colunas_autores = ['idProposicao', 'idDeputadoAutor', 'nomeAutor', 'siglaPartidoAutor', 'siglaUFAutor']

Ler os dados da proposições

In [4]:
df_array_proposicoes = []
caminho_dados = diretorio_dados + '/proposicoes-{ano}.csv'
for ano in anos:
    df_array_proposicoes.append(pd.read_csv(caminho_dados.format(ano=ano), sep=';', index_col="id"))

/home/guilherme/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Validar se os dados possuem consistência nas colunas para fazer join

In [8]:
igual_anterior = []
df_ant = df_array_proposicoes[0]
for i in range(0, len(df_array_proposicoes)):
    df = df_array_proposicoes[i]
    igual_anterior.append((df_ant.columns == df.columns).all())
    df_ant = df
(np.array(igual_anterior) == True).all()

True

In [6]:
df_proposicoes = pd.concat(df_array_proposicoes)[filtro_colunas_proposicoes]
df_proposicoes

,siglaTipo,ano,ementa,ementaDetalhada,keywords,dataApresentacao,ultimoStatus_dataHora,ultimoStatus_descricaoTramitacao,ultimoStatus_descricaoSituacao
id,,,,,,,,,
12863,INC,1998,"SUGERE AO PODER EXECUTIVO, POR INTERMEDIO DO M...",NaN,"SUGESTÃO, EXECUTIVO, (MC), (ECT), (TELEBRAS), ...",1998-05-12T00:00:00,1998-12-08T00:00:00,Recebimento na Mesa solicitando resposta,Arquivada
12881,INC,1998,"SUGERE AO PODER EXECUTIVO, POR INTERMEDIO DO M...",NaN,"SUGESTÃO, EXECUTIVO, (MC), (ECT), (TELEBRAS), ...",1998-05-28T00:00:00,1998-12-08T00:00:00,Recebimento na Mesa solicitando resposta,Arquivada
12936,INC,1998,"SUGERE AO PODER EXECUTIVO A IMPLEMENTAÇÃO, POR...",NaN,"AUTORIZAÇÃO, EXECUTIVO, IMPLEMENTAÇÃO, CONVENI...",1998-10-14T00:00:00,1998-11-19T00:00:00,Recebimento na Mesa solicitando resposta,Arquivada
12938,INC,1998,"SUGERE AO PODER EXECUTIVO , POR INTERMÉDIO DO ...",NaN,"SUGESTÃO, EXECUTIVO, (MF), ELABORAÇÃO, PROJETO...",1998-10-21T00:00:00,2002-11-28T00:00:00,Recebimento de Resposta,Arquivada
12940,INC,1998,"SUGERE AO EXECUTIVO , POR INTERMEDIO DO MINIST...",NaN,NaN,1998-10-27T00:00:00,1998-11-21T10:07:05,Publicação de Despacho,Arquivada
...,...,...,...,...,...,...,...,...,...
2337503,PL,2022,"Insere disposições nas Leis nos 4.737, de 15 d...",NaN,"Alteração, Código Eleitoral (1965), prevenção,...",2022-11-11T16:00:13,2022-11-11T16:00:13,Apresentação de Proposição,NaN
2337504,PL,2022,"Altera a Lei nº 7.565, de 19 de dezembro de 19...",NaN,"Alteração, Código Brasileiro de Aeronáutica, v...",2022-11-11T16:02:51,2022-11-11T16:02:51,Apresentação de Proposição,NaN
2337513,PL,2022,Institui a Lei Orgânica Nacional do Conselho T...,NaN,NaN,2022-11-11T16:38:27,2022-11-11T16:38:27,Apresentação de Proposição,NaN


Ler dados dos temas e autores

In [15]:
df_array_temas = []
caminho_dados_temas = diretorio_dados + '/proposicoesTemas-{ano}.csv'
for ano in anos:
    df_array_temas.append(pd.read_csv(caminho_dados_temas.format(ano=ano), sep=';'))

Filtrar linhas com idDeputadoAutor com valor NaN pois se referem a orgãos do governo e não pessoas individuais

In [3]:
def filtrar_e_preprocessar_df_autores(df_cru, ano):
    df_cru = df_cru[filtro_colunas_autores]
    df_cru.loc[:, 'ano'] = ano
    df_cru.loc[:, 'nomeAutor'] = df_cru.loc[:, 'nomeAutor'].str.upper()
    df_cru.loc[:, 'siglaPartidoAutor'] = df_cru.loc[:, 'siglaPartidoAutor'].str.upper()
    df_cru.loc[:, 'siglaUFAutor'] = df_cru.loc[:, 'siglaUFAutor'].str.upper()
    df_cru = df_cru[df_cru['idDeputadoAutor'].notna()]
    
    return df_cru

In [4]:
df_array_autores = []
caminho_dados_autores = diretorio_dados + '/proposicoesAutores-{ano}.csv'
for ano in anos:
    df = pd.read_csv(caminho_dados_autores.format(ano=ano), sep=';')
    df_proc = filtrar_e_preprocessar_df_autores(df, ano)
    df_array_autores.append(df_proc)

/home/guilherme/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/guilherme/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [5]:
df_autores = pd.concat(df_array_autores)
df_autores

,idProposicao,idDeputadoAutor,nomeAutor,siglaPartidoAutor,siglaUFAutor,ano
1,2092099,73896.0,FETTER JUNIOR,NaN,NaN,1998
2,2092098,73896.0,FETTER JUNIOR,NaN,NaN,1998
6,338763,73783.0,PADRE ROQUE,NaN,NaN,1998
7,335961,73760.0,MARISA SERRANO,NaN,NaN,1998
8,335960,74667.0,JOÃO FASSARELLA,NaN,NaN,1998
...,...,...,...,...,...,...
35836,2311331,160512.0,AUREO RIBEIRO,SOLIDARI,RJ,2022
35837,2053152,74283.0,VICENTINHO,PT,SP,2022
35838,996806,178876.0,CÉLIO SILVEIRA,PSDB,GO,2022
35839,587207,151208.0,LAERCIO OLIVEIRA,PR,SE,2022


A quantidade de projetos é 8700 vezes maior que o número de deputados. porém a grande maioria dos projetos tem autoria única

In [8]:
print('quantidade de deputados: ' + str(len(df_autores['idDeputadoAutor'].unique())))
print('quantidade de partidos: ' + str(len(df_autores['siglaPartidoAutor'].unique())))
print('quantidade de projetos: '+ str(len(df_autores['idProposicao'].unique())))

quantidade de deputados: 2139
quantidade de partidos: 58
quantidade de projetos: 505040


Selecionar projetos que tem mais de um autor. A maioria dos projetos possuem autoria única

In [9]:
df_aux = df_autores.groupby('idProposicao').size()
df_autores = df_autores[df_autores['idProposicao'].isin(df_aux[df_aux > 1].index)]
print('quantidade de projetos: '+ str(len(df_autores['idProposicao'].unique())))

quantidade de projetos: 30309


Partidos sem autores, nota-se que o `PCDB` é uma outra sigla para `PCDOB`

In [9]:
df_aux = df_autores.groupby('idProposicao').size()
part1 = set(df_autores[df_autores['idProposicao'].isin(df_aux[df_aux == 1].index)]['siglaPartidoAutor'].unique())
part2 = set(df_autores[df_autores['idProposicao'].isin(df_aux[df_aux > 1].index)]['siglaPartidoAutor'].unique())
part1 - part2

{'PCDB', 'PPR', 'PSTU', 'WWW'}

In [11]:
df_index = df_autores.set_index(['idProposicao', 'idDeputadoAutor'])
df_index

nomeAutor siglaPartidoAutor siglaUFAutor  \
idProposicao idDeputadoAutor                                                   
246488       73659.0              MARIA LAURA                PT           DF   
             74218.0          WALTER PINHEIRO                PT           BA   
246487       73659.0              MARIA LAURA                PT           DF   
             74218.0          WALTER PINHEIRO                PT           BA   
245954       74218.0          WALTER PINHEIRO                PT           BA   
...                                       ...               ...          ...   
2313780      206231.0               TED CONTI               PSB           ES   
             213274.0           RICARDO SILVA               PSB           SP   
             92172.0            MILTON COELHO               PSB           PE   
2311740      160575.0             ERIKA KOKAY                PT           DF   
             178825.0            LEO DE BRITO                PT           AC   

                               ano  
idProposicao idDeputadoAutor        
246488       73659.0          1998  
             74218.0          1998  
246487       73659.0          1998  
             74218.0          1998  
245954       74218.0          1998  
...                            ...  
2313780      206231.0         2022  
             213274.0         2022  
             92172.0          2022  
2311740      160575.0         2022  
             178825.0         2022  

[733368 rows x 4 columns]

Agrupar deputados que participaram de um mesmo projeto de dois a dois

In [12]:
from tqdm import tqdm
import itertools

dict_edges = {}
for date, new_df in tqdm(df_index.groupby(level=0)):
    for tup in itertools.combinations([j for (i,j) in new_df.index], 2):
        t = tup
        #ordenar
        if(t[0] > t[1]):
            t = (t[1], t[0])
        if(t in dict_edges):
            dict_edges[t].append(date)
        else:
            dict_edges[t] = [date]

len(dict_edges)

100%|████████████████████████████████████| 30309/30309 [01:04<00:00, 468.14it/s]


640958

Conversão de lista em string e string para lista

In [30]:
lista_str = ','.join(map(str,dict_edges[(73451.0, 74357.0)]))
lista_int = list(map(int, lista_str.split(',')))

print(lista_str, lista_int)

12896,22561 [12896, 22561]


Utilizando a conversão acima para transformar a lista de projetos em uma string. O número de arestas é muito alto comparado ao número de nós, o grafo é muito conectado

In [13]:
df_edges = pd.DataFrame([[key[0], 
                          key[1], 
                          ','.join(map(str,dict_edges[key])), 
                          len(dict_edges[key])] for key in dict_edges],
                       columns=['no1','no2','projetos', 'weight'])
df_edges

,no1,no2,projetos,weight
0,73451.0,74357.0,"12896,22561",2
1,73886.0,73910.0,"13492,15431,25095",3
2,74307.0,74694.0,13610,1
3,74218.0,74307.0,13610,1
4,74307.0,74544.0,13610,1
...,...,...,...,...
640953,215044.0,220008.0,2336964,1
640954,204504.0,220008.0,2337015,1
640955,179000.0,204480.0,2337357,1
640956,179000.0,204556.0,2337357,1


In [14]:
df_edges.to_csv(diretorio_dados_processados+"/arestas.csv", index=False)

In [15]:
import networkx as nx
gf = nx.from_pandas_edgelist(df_edges, source='no1', target='no2', edge_attr=True)

In [17]:
df_info_autores = df_autores.set_index('idDeputadoAutor')
df_info_autores.loc[74848]

,idProposicao,nomeAutor,siglaPartidoAutor,siglaUFAutor,ano
idDeputadoAutor,,,,,
74848.0,22126,JANDIRA FEGHALI,PCDOB,RJ,1998
74848.0,39521,JANDIRA FEGHALI,PCDOB,RJ,1999
74848.0,38340,JANDIRA FEGHALI,PCDOB,RJ,1999
74848.0,38339,JANDIRA FEGHALI,PCDOB,RJ,1999
74848.0,15225,JANDIRA FEGHALI,PCDOB,RJ,1999
...,...,...,...,...,...
74848.0,2314646,JANDIRA FEGHALI,PCDOB,RJ,2022
74848.0,2314567,JANDIRA FEGHALI,PCDOB,RJ,2022
74848.0,2314387,JANDIRA FEGHALI,PCDOB,RJ,2022


Fazer os dados de autores deduplicando

In [ ]:
def iteravel_para_string(iteravel):
    lista = list(iteravel)
    if(len(lista) == 1):
        return str(lista[0])
    if(np.nan in lista):
        lista.remove(np.nan)
    return ','.join(map(str,lista))

info_autores = []
for idAutor, new_df in tqdm(df_info_autores.groupby(level=0)):
    info = []
    info.append(idAutor)
    info.append(new_df.iloc[0, 1])
    partido = set()
    uf = set()
    for index, row in new_df.iterrows():
        partido.add(row['siglaPartidoAutor'])
        uf.add(row['siglaUFAutor'])
    info.append(iteravel_para_string(partido))
    info.append(iteravel_para_string(uf))
    info_autores.append(info)
    
info_autores

Feito dataframe com informações sobre deputados

In [37]:
df_deputados = pd.DataFrame(info_autores, columns=['id', 'nome', 'partidos', 'uf'])
df_deputados.to_csv(diretorio_dados_processados+'/deputados.csv')
df_deputados

,id,nome,partidos,uf
0,2995.0,LUIS CLAUDIO,nan,nan
1,3151.0,JAIRO ATAIDE,"PFL,DEM",MG
2,3155.0,BISPO GÊ TENUTA,"PFL,DEM",SP
3,4927.0,JORGINHO MALULY,"PFL,DEM",SP
4,4928.0,EDUARDO LOPES,"PRB,PSB",RJ
...,...,...,...,...
1879,219568.0,TIAGO ANDRINO,PSB,TO
1880,219585.0,MARCO BRASIL,PP,PR
1881,219592.0,LUCAS FOLLADOR,PSC,RO
1882,219599.0,SARGENTO ALEXANDRE,PODE,SP


Estranho mas existem deputados que mudaram de estado!

In [34]:
df_deputados[df_deputados['uf'].str.contains(',')]

,nome,partidos,uf
53,DR. ROSINHA,PT,"PA,PR"
200,ROBERTO FREIRE,PPS,"SP,PE"
256,PERPÉTUA ALMEIDA,"PPS,PCDOB","AC,AL"
295,ÁLVARO DIAS,"PDT,PSDB","RN,PR"
334,PAULO ROCHA,PT,"PA,PR"
420,WALTER PINHEIRO,PT,"PA,BA"
468,TATICO,"PL,PTB","DF,GO"
566,HUMBERTO COSTA,PT,"CE,PE"
809,CIDINHA CAMPOS,PDT,"RJ,PR"
846,VITOR PAULO,PRB,"RJ,DF"


In [39]:
nx.set_node_attributes(gf, df_deputados.set_index('id').to_dict('index'))

In [40]:
nx.write_gml(gf, diretorio_dados_processados+"/arestas.gml")

In [45]:
df_edges['weight'].describe()

count    640958.000000
mean         86.440658
std         131.132384
min           1.000000
25%          10.000000
50%          36.000000
75%         105.000000
max        1754.000000
Name: weight, dtype: float64

In [48]:
df_index = df_autores[df_autores['siglaPartidoAutor'].notna()]
df_index = df_index.set_index(['idProposicao', 'siglaPartidoAutor'])
df_index

idDeputadoAutor        nomeAutor siglaUFAutor  \
idProposicao siglaPartidoAutor                                                  
246488       PT                         73659.0      MARIA LAURA           DF   
             PT                         74218.0  WALTER PINHEIRO           BA   
246487       PT                         73659.0      MARIA LAURA           DF   
             PT                         74218.0  WALTER PINHEIRO           BA   
245954       PT                         74218.0  WALTER PINHEIRO           BA   
...                                         ...              ...          ...   
2313780      PSB                       206231.0        TED CONTI           ES   
             PSB                       213274.0    RICARDO SILVA           SP   
             PSB                        92172.0    MILTON COELHO           PE   
2311740      PT                        160575.0      ERIKA KOKAY           DF   
             PT                        178825.0     LEO DE BRITO           AC   

                                 ano  
idProposicao siglaPartidoAutor        
246488       PT                 1998  
             PT                 1998  
246487       PT                 1998  
             PT                 1998  
245954       PT                 1998  
...                              ...  
2313780      PSB                2022  
             PSB                2022  
             PSB                2022  
2311740      PT                 2022  
             PT                 2022  

[731924 rows x 4 columns]

Agrupar deputados que participaram de um mesmo projeto de dois a dois

In [49]:
from tqdm import tqdm
import itertools

dict_edges = {}
for date, new_df in tqdm(df_index.groupby(level=0)):
    partidos = set()
    for (i,j) in new_df.index:
        partidos.add(j)
    for tup in itertools.combinations(list(partidos), 2):
        t = tup
        #ordenar
        if(t[0] > t[1]):
            t = (t[1], t[0])
        if(t in dict_edges):
            dict_edges[t].append(date)
        else:
            dict_edges[t] = [date]

len(dict_edges)

100%|███████████████████████████████████| 30305/30305 [00:07<00:00, 4029.64it/s]


939

In [53]:
df_edges_partidos = pd.DataFrame([[key[0], 
                          key[1], 
                          ','.join(map(str,dict_edges[key])), 
                          len(dict_edges[key])] for key in dict_edges],
                       columns=['no1','no2','projetos', 'weight'])
df_edges_partidos

,no1,no2,projetos,weight
0,PCDOB,PDT,"12896,22561,26909,26956,28611,28614,28698,3379...",4784
1,PFL,PT,"13610,14944,26909,26956,26957,26973,27211,2721...",2250
2,PFL,PMDB,"13610,22240,26909,26956,26973,27209,27210,2721...",2452
3,PFL,PSB,"13610,26814,26815,26909,26973,27209,27210,2724...",1983
4,PCDOB,PFL,"13610,26909,26956,29155,33701,33852,36764,3739...",1796
...,...,...,...,...
934,PCDOB,UNIÃO,"2317290,2317424,2317472,2317508,2317654,231816...",80
935,PSOL,UNIÃO,"2317290,2317457,2317472,2317508,2317978,231835...",40
936,PATRIOTA,UNIÃO,"2317290,2317472,2317563,2317966,2317978,231798...",67
937,S.PART.,UNIÃO,2318119,1


In [55]:
gf_partidos = nx.from_pandas_edgelist(df_edges_partidos, source='no1', target='no2', edge_attr=True)
nx.write_gml(gf_partidos, diretorio_dados_processados+"/grafo_partidos.gml")